In [ ]:
# 10만개만 하려면 무슨 폴더를 가져와야하지?
import pandas as pd

df = pd.read_csv("./landr4_aggregate_20250117.csv")
print(len(df))
df

In [4]:
import re
from collections import Counter

df['folder'] = df['gcs_url'].apply(lambda x: x.split("/")[5])

cnt = Counter(list(df['folder']))

In [ ]:
df['gcs_url'][0]

In [ ]:
ndf = df[df['gcs_url'].str.contains("part_2") | df['gcs_url'].str.contains("part_3") | df['gcs_url'].str.contains("part_3")]
print(len(ndf))

In [7]:
ndf.to_csv("only_70000.csv")

In [ ]:
print(12)

In [7]:
import numpy as np

np.random.seed(42)  # 재현성을 위해 시드 고정
shuffled_indices = np.random.permutation(len(ndf))

split_index = int(len(ndf) * 0.9)
train_indices = shuffled_indices[:split_index]
test_indices = shuffled_indices[split_index:]

train_df = df.iloc[train_indices]
test_df = df.iloc[test_indices]

# 결과를 CSV로 저장
train_df.to_csv('./train_ld_23.csv', index=False)
test_df.to_csv('./valid_ld_23.csv', index=False)

In [ ]:
import re

ndf['audio_path'] = ndf['gcs_url'].apply(lambda x: re.sub('https://storage.cloud.google.com/clean-music', '/workspace', x))

In [ ]:
def build_prompt_v3(row):
    prompt = 'prompts : '
    prompt += row['tags'].lower() + ", " if not pd.isna(row['tags']) else ''
    prompt += row['genre'].lower() + ", " if not pd.isna(row['genre']) else ''
    prompt += row['type'].lower() + ", " if not pd.isna(row['type']) else ''
    prompt += 'key : ' + row['key'].lower() + ", " if not pd.isna(row['key']) else ''
    prompt += 'bpm : ' + f'{int(row["bpm"])}' + ", " if not pd.isna(row['bpm']) else ''
    return prompt[:-2]

# 각 행에 대해 build_prompt_v3 적용
ndf['generated_prompt'] = ndf.apply(build_prompt_v3, axis=1)

In [ ]:
import os
from scipy.io import wavfile
from pydub import AudioSegment
from tqdm import tqdm
import pandas as pd
from multiprocessing import Pool, cpu_count

def get_duration(file_path):
    try:
        if file_path.lower().endswith('.wav'):
            _, data = wavfile.read(file_path)
            duration = len(data) / 44100  # 기본 샘플레이트 44100Hz 가정
        else:
            audio = AudioSegment.from_file(file_path)
            duration = len(audio) / 1000.0  # 밀리초를 초로 변환
        return duration
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None

def process_files(file_paths):
    with Pool(16) as pool:
        durations = list(tqdm(pool.imap(get_duration, file_paths), total=len(file_paths)))
    return durations

ndf['exists'] = ndf['audio_path'].apply(lambda x: os.path.exists(x))
ndf = ndf[ndf['exists'] == True]

ndf['duration'] = process_files(ndf['audio_path'].tolist())

In [16]:
import numpy as np

np.random.seed(42)  # 재현성을 위해 시드 고정
shuffled_indices = np.random.permutation(len(ndf))

split_index = int(len(ndf) * 0.9)
train_indices = shuffled_indices[:split_index]
test_indices = shuffled_indices[split_index:]

train_df = ndf.iloc[train_indices]
test_df = ndf.iloc[test_indices]

# 결과를 CSV로 저장
train_df.to_csv('./train_ld_23.csv', index=False)
test_df.to_csv('./valid_ld_23.csv', index=False)

In [ ]:
# 10만개만 하려면 무슨 폴더를 가져와야하지?
import pandas as pd
import os
import re

df = pd.read_csv("./aggregate_20250122.csv")
print(len(df))
df['exist'] = df['audio_path'].apply(lambda x: os.path.exists(x))
df = df[df['exist']==True]
print(len(df))
# df = df[df['duration']<20.0]
# print(len(df))

df['audio_path'] = df['gcs_url'].apply(lambda x: re.sub('https://storage.cloud.google.com/clean-music/', '/workspace/', x))

df

In [ ]:
def build_prompt_v3(row):
    prompt = 'prompts : '
    prompt += row['tags'].lower() + ", " if not pd.isna(row['tags']) else ''
    prompt += row['genre'].lower() + ", " if not pd.isna(row['genre']) else ''
    prompt += row['type'].lower() + ", " if not pd.isna(row['type']) else ''
    prompt += 'key : ' + row['key'].lower() + ", " if not pd.isna(row['key']) else ''
    prompt += 'bpm : ' + f'{int(row["bpm"])}' + ", " if not pd.isna(row['bpm']) else ''
    return prompt[:-2]

# 각 행에 대해 build_prompt_v3 적용
df['generated_prompt'] = df.apply(build_prompt_v3, axis=1)

In [1]:
import os
from scipy.io import wavfile
from pydub import AudioSegment
from tqdm import tqdm
import pandas as pd
from multiprocessing import Pool, cpu_count
import librosa

def get_duration(file_path):
    try:
        audio, sr = librosa.load(file_path, sr=16000)
        duration = audio.shape[-1]/sr
        return duration
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None

def process_files(file_paths):
    with Pool(64) as pool:
        durations = list(tqdm(pool.imap(get_duration, file_paths), total=len(file_paths)))
    return durations

In [2]:
import pandas as pd

df1 = pd.read_csv('/workspace/csv-sample_landr_4_20250129.csv')
df2 = pd.read_csv('/workspace/csv-sample_sp_audioz_20250129.csv')
df3 = pd.read_csv('/workspace/csv-sample_sp_record_20250129.csv')

print(len(df1))
print(len(df2))
print(len(df3))

754452
216979
525919


In [3]:
import re

df1['audio_path'] = df1['gcs_url'].apply(lambda x: re.sub('https://storage.cloud.google.com/clean-music/', '/workspace/', x))
df1['exist'] = df1['audio_path'].apply(lambda x: os.path.exists(x))
df1 = df1[df1['exist'] == True]
print(len(df1))
df1['duration'] = process_files(df1['audio_path'].tolist())

751754


  0%|          | 1/751754 [00:01<370:27:48,  1.77s/it][src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
  1%|          | 5247/751754 [00:04<04:59, 2488.57it/s][src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[s

Error processing file /workspace/landr_4/part_9/packs-baby-trap/SC155 04 Beat 128bpm Em Melodic Layered 01.mp3: 


[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the r

Error processing file /workspace/landr_4/part_13/packs-no-time-vol-1/NT1_4_122bpm_Hihat 1.mp3: 


...

 87%|████████▋ | 656155/751754 [05:32<00:26, 3547.99it/s][src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
 88%|████████▊ | 659634/751754 [05:33<00:20, 4542.24it/s][src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of t

Error processing file /workspace/landr_4/part_16/packs-newdeal-techno/QR_Shake_9.mp3: 


 94%|█████████▍| 708458/751754 [05:51<00:16, 2603.29it/s][src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
 94%|█████████▍| 708737/751754 [05:51<00:22, 1909.86it

Error processing file /workspace/landr_4/part_15/packs-underground-hip-hop-drums-the-add-on/ESM_Underground_Hat_One_Shot_Tape_Tek_Boombap_Hi.mp3: 


[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
[src/libmpg123/id3.c:INT123_parse_new_id3():1073] error: ID3v2: non-syncsafe size of CTOC frame, skipping the remainder of tag
100%|██████████| 751754/751754 [06:12<00:00, 2015.75it/s]


In [4]:
print(len(df1))
df1 = df1[~df1['duration'].isna()]
print(len(df1))

751754
751747


In [5]:
df1

,pack_title,sample_title,gcs_url,tags,time,type,bpm,key,genre,chord,audio_path,exist,duration
0,packs-global-rnb,Track_01_117bpm_Bass_D Minor,https://storage.cloud.google.com/clean-music/l...,Bass,0:08,Loop,117.0,D minor,"HIP-HOP, R&B","Time:0.00-2.50/Chord:N, Time:2.50-4.40/Chord:F...",/workspace/landr_4/part_2/packs-global-rnb/Tra...,True,8.205125
1,packs-global-rnb,Track_01_117bpm_Drum Fill_D Minor,https://storage.cloud.google.com/clean-music/l...,"Drums, Fills",0:08,Loop,117.0,D minor,"HIP-HOP, R&B",Time:0.00-8.30/Chord:N,/workspace/landr_4/part_2/packs-global-rnb/Tra...,True,8.205125
2,packs-global-rnb,Track_01_117bpm_Drum Loop_D Minor,https://storage.cloud.google.com/clean-music/l...,Drums,0:08,Loop,117.0,D minor,"HIP-HOP, R&B",Time:0.00-8.30/Chord:N,/workspace/landr_4/part_2/packs-global-rnb/Tra...,True,8.205125
3,packs-global-rnb,Track_01_117bpm_Fx_D Minor,https://storage.cloud.google.com/clean-music/l...,FX,0:08,Loop,117.0,D minor,"HIP-HOP, R&B",Time:0.00-8.30/Chord:N,/workspace/landr_4/part_2/packs-global-rnb/Tra...,True,8.205125
4,packs-global-rnb,Track_01_117bpm_Kick_D Minor,https://storage.cloud.google.com/clean-music/l...,"Kick, Drums",0:08,Loop,117.0,D minor,"HIP-HOP, R&B",--,/workspace/landr_4/part_2/packs-global-rnb/Tra...,True,8.205125
...,...,...,...,...,...,...,...,...,...,...,...,...,...
754447,packs-urban-dirty-drums-one-shots,MCT-UDOS-Hi_Hat_109,https://storage.cloud.google.com/clean-music/l...,"Hi-Hat, Drums",0:01,One Shot,NaN,NaN,"HIP-HOP, LO-FI",--,/workspace/landr_4/part_15/packs-urban-dirty-d...,True,0.457625
754448,packs-urban-dirty-drums-one-shots,MCT-UDOS-Hi_Hat_11,https://storage.cloud.google.com/clean-music/l...,"Hi-Hat, Drums",0:01,One Shot,NaN,NaN,"HIP-HOP, LO-FI",--,/workspace/landr_4/part_15/packs-urban-dirty-d...,True,0.680688
754449,packs-urban-dirty-drums-one-shots,MCT-UDOS-Hi_Hat_110,https://storage.cloud.google.com/clean-music/l...,"Hi-Hat, Drums",0:01,One Shot,NaN,NaN,"HIP-HOP, LO-FI",--,/workspace/landr_4/part_15/packs-urban-dirty-d...,True,0.176187
754450,packs-urban-dirty-drums-one-shots,MCT-UDOS-Hi_Hat_111,https://storage.cloud.google.com/clean-music/l...,"Hi-Hat, Drums",0:01,One Shot,NaN,NaN,"HIP-HOP, LO-FI",--,/workspace/landr_4/part_15/packs-urban-dirty-d...,True,0.498000


In [11]:
df2 = pd.read_csv('/workspace/audioz_0205.csv')
df3 = pd.read_csv('/workspace/sp_record_20250205.csv')

In [10]:
from tqdm import tqdm

allds = []

for i in tqdm(range(len(df1))):
    d = df1.iloc[i]
    prompt = build_prompt_v3(d)

    allds.append({
        'generated_prompt': prompt,
        'duration': d['duration'],
        'audio_path': d['audio_path']
    })

100%|██████████| 751747/751747 [00:49<00:00, 15162.90it/s]


In [20]:
for i in tqdm(range(len(df2))):
    d = df2.iloc[i]
    prompt = build_prompt_v3(d)
    
    allds.append({
        'generated_prompt': prompt,
        'duration': d['duration'],
        'audio_path': d['audio_path']
    })

100%|██████████| 216956/216956 [00:15<00:00, 13897.17it/s]


In [18]:
def build_prompt_v3(row):
    prompt = 'prompts : '
    prompt += row['instruments'].lower() + ", " if not pd.isna(row['instruments']) else ''
    prompt += row['tags'].lower() + ", " if not pd.isna(row['tags']) else ''
    prompt += row['genre'].lower() + ", " if not pd.isna(row['genre']) else ''
    prompt += 'loop, ' if row['is_loops'] == True else 'one shot, '
    prompt += 'key : ' + row['key'].lower() + ", " if not pd.isna(row['key']) and row['bpm'] != '--' else ''
    prompt += 'bpm : ' + f'{int(row["bpm"])}' + ", " if not pd.isna(row['bpm']) and row['bpm'] != '--' else ''
    return prompt[:-2]

In [26]:
def build_prompt_v4(row):
    prompt = 'prompts : '
    prompt += row['tags'].lower() + ", " if not pd.isna(row['tags']) else ''
    prompt += row['oneloop'] + ", "
    prompt += 'key : ' + row['key'].lower() + ", " if not pd.isna(row['key']) and row['bpm'] != '--' else ''
    prompt += 'bpm : ' + f'{int(row["bpm"])}' + ", " if not pd.isna(row['bpm']) and row['bpm'] != '--' else ''
    return prompt[:-2]

In [27]:
build_prompt_v4(df3.iloc[10])

'prompts : drums, snares, future bass, tearout dubstep, trap edm, oneshot'

In [28]:
for i in tqdm(range(len(df3))):
    d = df3.iloc[i]
    prompt = build_prompt_v4(d)
    
    allds.append({
        'generated_prompt': prompt,
        'duration': d['duration'],
        'audio_path': d['audio_path']
    })

100%|██████████| 517680/517680 [00:32<00:00, 15949.38it/s]


In [29]:
ndf = pd.DataFrame(allds)
print(len(ndf))
ndf

1486649


,generated_prompt,duration,audio_path
0,"prompts : bass, hip-hop, r&b, loop, key : d mi...",8.205125,/workspace/landr_4/part_2/packs-global-rnb/Tra...
1,"prompts : drums, fills, hip-hop, r&b, loop, ke...",8.205125,/workspace/landr_4/part_2/packs-global-rnb/Tra...
2,"prompts : drums, hip-hop, r&b, loop, key : d m...",8.205125,/workspace/landr_4/part_2/packs-global-rnb/Tra...
3,"prompts : fx, hip-hop, r&b, loop, key : d mino...",8.205125,/workspace/landr_4/part_2/packs-global-rnb/Tra...
4,"prompts : kick, drums, hip-hop, r&b, loop, key...",8.205125,/workspace/landr_4/part_2/packs-global-rnb/Tra...
...,...,...,...
1486644,"prompts : percussion, house, techno, melodic t...",0.265625,/workspace/sp/record_20/ZEN_TIF_percussion_one...
1486645,"prompts : house, techno, percussion, shakers, ...",0.156812,/workspace/sp/record_20/ZEN_TIF_shaker_one_sho...
1486646,"prompts : house, techno, drums, claps, melodic...",0.285813,/workspace/sp/record_20/ZEN_TIF_clap_one_shot_...
1486647,"prompts : drums, snares, house, techno, melodi...",0.257125,/workspace/sp/record_20/ZEN_TIF_snare_one_shot...


In [31]:
import numpy as np

np.random.seed(40)  # 재현성을 위해 시드 고정
shuffled_indices = np.random.permutation(len(ndf))
split_index = int(len(ndf) * 0.95)
train_indices = shuffled_indices[:split_index]
test_indices = shuffled_indices[split_index:]

train_df = ndf.iloc[train_indices]
test_df = ndf.iloc[test_indices]
print(len(train_df))

train_df.to_csv('./train_0205.csv', index=False)
test_df.to_csv('./valid_0205.csv', index=False)

1412316


In [32]:
ndf['exist'] = ndf['audio_path'].apply(lambda x: os.path.exists(x))
print(ndf[ndf['exist'] == True])

                                          generated_prompt  duration  \
0        prompts : bass, hip-hop, r&b, loop, key : d mi...  8.205125   
1        prompts : drums, fills, hip-hop, r&b, loop, ke...  8.205125   
2        prompts : drums, hip-hop, r&b, loop, key : d m...  8.205125   
3        prompts : fx, hip-hop, r&b, loop, key : d mino...  8.205125   
4        prompts : kick, drums, hip-hop, r&b, loop, key...  8.205125   
...                                                    ...       ...   
1486644  prompts : percussion, house, techno, melodic t...  0.265625   
1486645  prompts : house, techno, percussion, shakers, ...  0.156812   
1486646  prompts : house, techno, drums, claps, melodic...  0.285813   
1486647  prompts : drums, snares, house, techno, melodi...  0.257125   
1486648  prompts : drums, snares, house, techno, melodi...  0.463000   

                                                audio_path  exist  
0        /workspace/landr_4/part_2/packs-global-rnb/Tra...   True  

In [33]:
print(len(test_df))

74333
